# SparkSession
A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files.
The entry point to programming Spark with the Dataset and DataFrame API.

In [6]:
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [8]:
spark = (SparkSession.builder.appName("pyspark-dataframe-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
# spark.sparkContext.getConf().getAll()

23/10/09 19:40:04 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NullPointerException
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:668)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(Construc

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-dataframe-demo-2023-10-09 10:38:50.146724>

# DataFrame
A distributed collection of data grouped into named columns

## From list of tuples, dictionary

In [4]:
l = [("AIDE", "FDE", 3)]
spark.createDataFrame(l).show()

+----+---+---+
|  _1| _2| _3|
+----+---+---+
|AIDE|FDE|  3|
+----+---+---+



In [5]:
spark.createDataFrame(l, ["institute", "course", "number"]).show()

+---------+------+------+
|institute|course|number|
+---------+------+------+
|     AIDE|   FDE|     3|
+---------+------+------+



In [6]:
import pandas as pd
pd.DataFrame([{'institute': 'AIDE', 'course': 'FDE', 'number': 3}])

,institute,course,number
0,AIDE,FDE,3


In [7]:
d = [{"name": "Alice", "age": 1}]
spark.createDataFrame(d).show()

+---+-----+
|age| name|
+---+-----+
|  1|Alice|
+---+-----+



## From RDDs

In [ ]:
l = [("Alice", 1)]
rdd = sc.parallelize(l)
spark.createDataFrame(rdd).collect()

In [ ]:
# with list of column names
df = spark.createDataFrame(rdd, ["name", "age"])
df.collect()

In [ ]:
# with Row definition
from pyspark.sql import Row
Person = Row("name", "age")
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.collect()

In [ ]:
df2.printSchema()

In [ ]:
# with schema definition
from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
  ])
df3 = spark.createDataFrame(rdd, schema)
df3.collect()

In [ ]:
df3.printSchema()

In [ ]:
# with string definition, New in version 2.0.
rdd = sc.parallelize(l)
print(spark.createDataFrame(rdd, "a: string, b: int").collect())

rdd = rdd.map(lambda row: row[1])
print(spark.createDataFrame(rdd, "int").collect())

## From pandas

In [ ]:
import pandas
print(spark.createDataFrame(df.toPandas()).collect())
print(spark.createDataFrame(pandas.DataFrame([["Alice", 2]])).collect())

# SQLContext

In [ ]:
df.show()

In [ ]:
# New in version 2.0
df.createOrReplaceTempView("table1")
df2 = spark.sql("SELECT name as N, age as A from table1")
df2.show()

In [ ]:
sqlContext.registerDataFrameAsTable(df, "table1")
sqlContext.registerDataFrameAsTable(df2, "table2")
sqlContext.tableNames()


In [ ]:
sqlContext.tables().show()

In [ ]:
df3 = sqlContext.tables()
df3.filter("tableName = 'table1'").show()

In [ ]:
sqlContext.dropTempTable("table1")
sqlContext.dropTempTable("table2")

In [ ]:
sqlContext.tableNames()

## UDF: User Defined Function

In [ ]:
sqlContext.registerFunction("stringLengthString", lambda x: len(x))
sqlContext.sql("SELECT stringLengthString('test')").collect()

In [ ]:
from pyspark.sql.types import IntegerType
sqlContext.registerFunction("stringLengthInt", lambda x: len(x), IntegerType())
sqlContext.sql("SELECT stringLengthInt('test')").collect()

In [ ]:
sqlContext.udf.register("stringLengthInt", lambda x: len(x), IntegerType())
sqlContext.sql("SELECT stringLengthInt('test')").collect()

# Working with DataFrame

In [ ]:
l = [("Alice", 2, 12), ("Bob", 5, 25)]
rdd = sc.parallelize(l)
df = sqlContext.createDataFrame(rdd, "name: string, age: int, height: int")
df.show()

In [ ]:
df.createOrReplaceTempView("people")

df2 = spark.sql("select * from people")
df2.show()

In [ ]:
df.repartition(10).rdd.getNumPartitions()

In [ ]:
data = df.union(df).repartition("age")
data.show()

In [ ]:
data = data.repartition(7, "age")
data.show()

In [ ]:
data.rdd.getNumPartitions()

In [ ]:
data = data.repartition("name", "age")
data.show()

In [ ]:
# withColumn(colName, col)
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name.
df.withColumn("age2", df.age + 2).show()

In [ ]:
df.withColumnRenamed("age", "age2").show()

In [ ]:
df.select(df.age.cast("string").alias("ages")).show()

In [ ]:
df.select(df.age.cast(StringType()).alias("ages")).show()

## Aggregate
Aggregate on the entire DataFrame without groups (shorthand for df.groupBy.agg()).

In [ ]:
df.agg({"age": "max"}).show()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.age)).show()

In [ ]:
gdf = df.groupBy(df.name)
gdf.agg({"*": "count"}).show()

In [ ]:
from pyspark.sql import functions as F
gdf.agg(F.min(df.age)).show()

In [ ]:
gdf.agg(F.min(df.age)).explain()

## Alias

In [ ]:
from pyspark.sql.functions import *
df_as1 = df.alias("df_as1")
df_as2 = df.alias("df_as2")
joined_df = df_as1.join(df_as2, col("df_as1.name") == col("df_as2.name"), "inner")
joined_df = joined_df.select("df_as1.name", "df_as2.name", "df_as2.age")
joined_df.toPandas()

In [ ]:
joined_df.explain()

## Stats

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.schema

In [ ]:
df.storageLevel

In [ ]:
df.count()

In [ ]:
df.groupBy().sum("age").show()

In [ ]:
df.groupBy().sum("age", "height").show()

In [ ]:
df.groupBy().avg("age").show()

In [ ]:
df.groupBy().avg("age", "height").show()

In [ ]:
df.columns

In [ ]:
df.name

In [ ]:
df["name"]

In [ ]:
df.age + 1

In [ ]:
# cube(*col): Create a multi-dimensional cube for the current DataFrame using the specified columns, so we can run aggregation on them.
df.cube("name", df.age).count().orderBy("name", "age").show()

In [ ]:
df.describe(["age"]).show()

In [ ]:
df.describe().show()

In [ ]:
df.distinct().count()

In [ ]:
df.dtypes

In [ ]:
df.explain()

In [ ]:
df.explain(True)

In [ ]:
df.groupBy().avg().show()

In [ ]:
df.groupBy("name").agg({"age": "mean"}).show()

In [ ]:
df.groupBy(df.name).avg().show()

In [ ]:
df.groupBy(["name", df.age]).count().show()

In [ ]:
df.groupBy().max("age").show()

In [ ]:
df.groupBy().max("age", "height").show()

In [ ]:
df.groupBy().mean("age").show()

In [ ]:
df.groupBy().mean("age", "height").show()

## Join

In [ ]:
df.select("age", "name").show()

In [ ]:
df2.select("name", "height").show()

In [ ]:
df.drop("age").show()

In [ ]:
df.drop(df.age).show()

In [ ]:
df.join(df2, df.name == df2.name, "inner").drop(df.name).drop(df.age).show()

In [ ]:
df.join(df2, "name", "inner").drop("age", "height").show()

In [ ]:
from pyspark.sql import Row
df = sc.parallelize([
    Row(name="Alice", age=5, height=80),
    Row(name="Alice", age=5, height=80),
    Row(name="Alice", age=10, height=80)
  ]).toDF()
df.dropDuplicates().show()

In [ ]:
df.dropDuplicates(["name", "height"]).show()

In [ ]:
df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height).show()

In [ ]:
df.join(df2, 'name', 'outer').select('name', df.height).show()

In [ ]:
cond = [df.name == df2.name, df.age == df2.age]
df.join(df2, cond, 'outer').select(df.name, df2.age).show()

In [ ]:
df.join(df2, 'name').select(df.name, df2.height).show()

In [ ]:
df.join(df2, ['name', 'age']).select(df.name, df.age).show()

## Filter

In [ ]:
l = [("Alice", 2, 12), ("Bob", 5, 25)]
rdd = sc.parallelize(l)
df = sqlContext.createDataFrame(rdd, "name: string, age: int, height: int")
df.show()


In [ ]:
df.filter(df.age > 3).show()

In [ ]:
df.filter("age > 3").show()

In [ ]:
df.where("age=2").show()

In [ ]:
df.first()

In [ ]:
df.head()

In [ ]:
df.limit(1).collect()

In [ ]:
df.limit(0).collect()

In [ ]:
# orderBy
print(df.sort(df.age.desc()).collect())
print(df.sort("age", ascending=False).collect())
print(df.orderBy(df.age.desc()).collect())

from pyspark.sql.functions import *
print(df.sort(asc("age")).collect())
print(df.sort(desc("age"), "name").collect())
print(df.orderBy(["age", "name"], ascending=[0, 1]).collect())

In [ ]:
print(df.filter(df.name.endswith("ice")).collect())
print(df.filter(df.name.endswith("ice$")).collect())

In [ ]:
# get subfield RDD > RDD, gets a field by name in a StructField.
from pyspark.sql import Row
df1 = sc.parallelize([Row(r=Row(a=1, b="b"))]).toDF()
df1.show()

In [ ]:
df1.select(df1.r.getField("b")).show()
df1.select(df1.r.getField("a")).show()

In [ ]:
# RDD contains list and dictionary
df1 = sc.parallelize([([1, 2], {"key": "value"})]).toDF(["l", "d"])
df1.show()

In [ ]:
df1.select(df1.l.getItem(0), df1.d.getItem("key")).show()
df1.select(df1.l[0], df1.d["key"]).show()

In [ ]:
from pyspark.sql import Row
df1 = sc.parallelize([Row(name=u"Tom", height=80), Row(name=u"Alice", height=None)]).toDF()
df1.show()

In [ ]:
print(df1.filter(df1.height.isNotNull()).collect())
print(df1.filter(df1.height.isNull()).collect())

In [ ]:
print(df[df.name.isin("Bob", "Mike")].collect())
print(df[df.age.isin(1, 2, 3)].collect())

In [ ]:
df.filter(df.name.like("Al%")).collect()

In [ ]:
from pyspark.sql import functions as F
df.select(df.name, F.when(df.age > 3, 1).otherwise(0)).show()